In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"  # use v0.3 if you prefer
# tok = AutoTokenizer.from_pretrained(MODEL_ID)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     device_map="auto",
#     load_in_4bit=True,
#     torch_dtype=torch.float16
# )

# def chat(messages, max_new_tokens=240, temperature=0.7, top_p=0.9, repetition_penalty=1.1):
#     text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     inputs = tok([text], return_tensors="pt").to(model.device)
#     out = model.generate(
#         **inputs,
#         max_new_tokens=max_new_tokens,
#         do_sample=True,
#         temperature=temperature,
#         top_p=top_p,
#         repetition_penalty=repetition_penalty
#     )
#     return tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
#MYSTRAL 7B

In [ ]:
# !pip -q install "transformers>=4.43.0" "accelerate>=0.33.0" bitsandbytes sentencepiece

# import torch, json, re
# from transformers import AutoTokenizer, AutoModelForCausalLM

# MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# tok = AutoTokenizer.from_pretrained(MODEL_ID)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     device_map="auto",            # GPU if available, else CPU
#     load_in_4bit=True,            # 4-bit quantization
#     torch_dtype=torch.float16
# )

# def chat(messages, max_new_tokens=220, temperature=0.7, top_p=0.9, repetition_penalty=1.1):
#     text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     inputs = tok([text], return_tensors="pt").to(model.device)
#     out = model.generate(
#         **inputs,
#         max_new_tokens=max_new_tokens,
#         do_sample=True,
#         temperature=temperature,
#         top_p=top_p,
#         repetition_penalty=repetition_penalty,
#         eos_token_id=tok.eos_token_id,
#     )
#     return tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, json, re

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

def chat(messages, max_new_tokens=220, temperature=0.6, top_p=0.9, repetition_penalty=1.1):
    # build chat prompt
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            eos_token_id=tok.eos_token_id,
        )
    # decode only the new tokens
    gen = out[0][inputs["input_ids"].shape[1]:]
    resp = tok.decode(gen, skip_special_tokens=True)
    return resp


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
test_resp = chat(
    [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Say hello in one short sentence."}
    ],
    max_new_tokens=50
)
print("TEST RAW:", repr(test_resp))


TEST RAW: '"Hello!"'


In [ ]:
SYS_PROMPT = (
  "You write short, polite B2B cold emails.\n"
  "Rules:\n"
  "- 60 to 120 words.\n"
  "- One clear call to action.\n"
  "- No links in the first email.\n"
  "- Use ONLY the information from lead_profile. Do not invent facts.\n"
  "- Use greeting and closing from style_profile.\n"
  "Return JSON only: {\"subject\":\"...\",\"body\":\"...\"}.\n"
)


In [ ]:
MBTI_TONE = {
 "ENTJ":{"assertiveness":"high","formality":"high","avg_wps":18,"cta_style":"decisive"},
 "INTJ":{"assertiveness":"med-high","formality":"high","avg_wps":17,"cta_style":"analytical"},
 "ENFJ":{"assertiveness":"medium","formality":"medium","avg_wps":20,"cta_style":"collaborative"},
 "ENFP":{"assertiveness":"medium","formality":"medium","avg_wps":21,"cta_style":"story"},
}
DEFAULT_TONE = {"assertiveness":"medium","formality":"medium","avg_wps":19,"cta_style":"consultative"}

def mbti_to_style_profile(mbti: str):
    base = MBTI_TONE.get((mbti or "").upper(), DEFAULT_TONE)
    return {
        "style_name": f"MBTI:{mbti or 'Consultative'}",
        "avg_words": 95,
        "avg_wps": base["avg_wps"],
        "formality": base["formality"],
        "assertiveness": base["assertiveness"],
        "cta_style": base["cta_style"],
        "greeting": "Hi {first_name},",
        "closing": "Best regards,\n{sender_name}",
        "quirks": []
    }

def normalize_profile(j):
    j = {**{"objective":"", "optional_context":"", "profile_data":{}}, **j}
    p = j["profile_data"]
    p.setdefault("name","")
    p.setdefault("headline","")
    p.setdefault("career_history",[])
    p.setdefault("skills",[])
    p.setdefault("recent_activity",[])
    p.setdefault("company_updates",[])
    p.setdefault("technical_contributions",[])
    p.setdefault("inferred_mbti","")
    return j

def build_lead_profile(j):
    p = j["profile_data"]
    role = p["career_history"][0]["role"] if p.get("career_history") else ""
    company = p["career_history"][0]["company"] if p.get("career_history") else ""
    return {
      "name": p.get("name",""),
      "headline": p.get("headline",""),
      "role": role,
      "company": company,
      "skills": p.get("skills", [])[:8],
      "recent_activity": p.get("recent_activity", [])[:3],
      "company_updates": p.get("company_updates", [])[:2],
      "technical_contributions": p.get("technical_contributions", [])[:3],
      "objective": j.get("objective",""),
      "optional_context": j.get("optional_context","")
    }

def generate_email(user_json, sender_name="Team", address="Your Address"):
    j = normalize_profile(user_json)
    lead_profile = build_lead_profile(j)
    mbti = j["profile_data"].get("inferred_mbti","")
    style_profile = mbti_to_style_profile(mbti)

    user_prompt = (
      "lead_profile = " + json.dumps(lead_profile, ensure_ascii=False) + "\n" +
      "style_profile = " + json.dumps(style_profile, ensure_ascii=False)
    )

    raw = chat(
        [
          {"role":"system", "content": SYS_PROMPT},
          {"role":"user", "content": user_prompt},
        ],
        max_new_tokens=220,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    print("RAW MODEL OUTPUT:\n", raw, "\n--- END RAW ---")  # DEBUG

    # Try to parse JSON
    email = {"subject":"", "body":"", "raw": raw}
    try:
        # if it already looks like pure JSON
        txt = raw.strip()
        if txt.startswith("{") and txt.endswith("}"):
            parsed = json.loads(txt)
        else:
            # extract first {...}
            m = re.search(r'\{.*\}', txt, re.S)
            if m:
                parsed = json.loads(m.group(0))
            else:
                parsed = {}
        email["subject"] = parsed.get("subject","").strip()
        email["body"] = parsed.get("body","").strip()
    except Exception as e:
        print("JSON parse failed, falling back to raw text:", e)

    # Fallback: if body still empty, just use raw
    if not email["body"].strip():
        email["subject"] = email["subject"] or "Quick idea for you"
        email["body"] = raw.strip()

    # Add signature/compliance
    if email["body"] and "unsubscribe" not in email["body"].lower():
        email["body"] += f"\n\n—\n{address}\nIf this isn’t relevant, reply “unsubscribe” and we won’t contact you again."
    email["body"] = email["body"].replace("{sender_name}", sender_name)
    return email


In [ ]:
user_json = {
  "objective": "Explore solutions for post-signup activation drop.",
  "optional_context": "We can align with their UPI rollout.",
  "profile_data": {
    "name": "Priya Iyer",
    "headline": "Head of Growth | Fintech",
    "career_history": [{"role":"Head of Growth","company":"Acme Fintech","duration":"2021–present","description":"Scaled lifecycle programs."}],
    "skills": ["Lifecycle","Activation","CRM","Experimentation"],
    "recent_activity": ["Onboarding friction and paywall timing discussion."],
    "company_updates": ["Launched UPI autopay in Q3."],
    "technical_contributions": [],
    "inferred_mbti": "ENTJ"
  }
}

email = generate_email(user_json, sender_name="Sumit", address="123 OMR, Chennai")

print("SUBJECT:", email["subject"])
print("BODY:\n", email["body"])


RAW MODEL OUTPUT:
 Sure, here's a sample template for a short, polite B2B cold email that follows the guidelines mentioned above:

Subject: [Insert Company Name] - Call to Action

Dear [First Name],

I hope this email finds you well. I am writing to reach out to you as part of our ongoing efforts to grow our client base.

As per your recent engagement with us, we have identified a need for [specific product/service] in your business. We believe that our expertise in [industry] and [unique selling proposition] would be a perfect fit for your team.

[Insert Company Name] is committed to providing high-quality services that meet the needs of our clients. Our team members possess the skills and experience required to provide [insert specific benefits]. We offer competitive rates and flexible payment plans to ensure that your budget doesn't affect the outcome of your project.

To make things simpler for both parties, we will work closely with you to understand your requirements in detail 
-